In [1]:
import pandas as pd
import numpy as np


In [43]:
genome = pd.read_csv('genome.csv', on_bad_lines='skip')
genome.rename(columns={'symbol':'gene'}, inplace=True)
slim_genome = genome[['gene','#"chrom"', 'chromStart', 'chromEnd', 'location']]
slim_genome.rename(columns={'#"chrom"':'chrom_num'}, inplace=True)
mask = slim_genome['chrom_num'].apply(lambda x: '_' not in x)
slimmer_genome = slim_genome[mask]
slimmer_genome


<ipython-input-43-4cbf0809cc1c>:1: DtypeWarning: Columns (26,39,44,50,56) have mixed types. Specify dtype option on import or set low_memory=False.
  genome = pd.read_csv('genome.csv', on_bad_lines='skip')
<ipython-input-43-4cbf0809cc1c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_genome.rename(columns={'#"chrom"':'chrom_num'}, inplace=True)


,gene,chrom_num,chromStart,chromEnd,location
0,DDX11L1,chr1,11873,14408,1p36.33
1,WASH7P,chr1,14361,29369,1p36.33
2,MIR6859-1,chr1,17368,17435,1p36.33
3,MIR1302-2HG,chr1,29773,35417,1p36.33
4,MIR1302-2,chr1,30365,30502,1p36.33
...,...,...,...,...,...
38280,SHANK3,chr22,50672822,50733211,22q13.33
38281,RNU6-409P,chr22,50691259,50691362,22q13.33
38282,ACR,chr22,50738203,50745338,22q13.33
38283,RPL23AP82,chr22,50757085,50799636,22q13.33


In [50]:
ccle_data = pd.read_csv('/content/tnbc_cn_melted.csv')
ccle_data
ccle_mapped = pd.merge(ccle_data, slimmer_genome, on='gene', how='left')

In [51]:
#This is the simplified version (doens't take in gene expresion, ampID, or classificaition )
def interval_finder(dataframe, cell_line: str):
  cn_groupby = dataframe.groupby('cn')
  new_dict = {'interval_size_mbp': [],
              'locations': [],
              'genes': [],
              'copy_number': [],
              'interval':[],
              }
  for cn, df in cn_groupby:
    first_instance = df.iloc[0]
    chr_groupby = df.groupby('chrom_num')
    interval_list = []
    total_interval_length = 0
    for chr_num, df in chr_groupby:
      interval_start = df['chromStart'].min()
      interval_end = df['chromEnd'].max()
      try:
        current_interval_length = ((interval_end-interval_start)/1000000)
      except UnboundLocalError:
        currenth_interval_length = 0

      interval_list.append(f"{chr_num}: {interval_start}-{interval_end}")
      total_interval_length += current_interval_length

    locations = list(df['location'].unique())
    genes = list(set(df['gene']))
    new_dict['interval'].append(interval_list)
    new_dict['interval_size_mbp'].append(total_interval_length)
    new_dict['locations'].append(locations)
    new_dict['genes'].append(genes)
    new_dict['copy_number'].append(cn)
  new_df = pd.DataFrame(new_dict)
  new_df['cell_line'] = cell_line
  return new_df.sort_values('copy_number', ascending=False)



In [52]:
interval_finder(ccle_mapped[ccle_mapped['cell_line']=='SUM159PT'], 'SUM159PT')

,interval_size_mbp,locations,genes,copy_number,interval,cell_line
112,1.070314,[8q24.21],"[MIR1205, CASC8, MYC, MIR1207, CASC11, MIR1208...",4.248059e+00,[chr8: 127079873.0-128150187.0],SUM159PT
111,0.039086,[5p15.33],"[LSINCT5, IRX2]",4.233245e+00,[chr5: 2712590.0-2751676.0],SUM159PT
110,0.007860,[8q24.21],[PCAT1],3.395614e+00,[chr8: 127013153.0-127021013.0],SUM159PT
109,0.328545,[8q24.21],[CCDC26],2.581853e+00,[chr8: 129351693.0-129680238.0],SUM159PT
108,1.345366,[11q22.3],"[CWF19L2, GUCY1A2, LINC02719]",1.753690e+00,[chr11: 106112458.0-107457824.0],SUM159PT
...,...,...,...,...,...,...
4,0.159608,[Yq11.221],[USP9Y],5.077056e-06,[chrY: 12701230.0-12860838.0],SUM159PT
3,4.124897,"[Yp11.2, Yp11.31]","[ZFY, TTTY1B, PCDH11Y, LINC00278, AMELY, TSPY2...",2.233320e-09,[chrY: 2786854.0-6911751.0],SUM159PT
2,0.377798,[Yp11.2],"[TSPY3, TSPY1, TSPY8, FAM197Y6, TSPY10, FAM197...",1.944408e-09,[chrY: 9337463.0-9715261.0],SUM159PT
1,5.903271,"[Yq11.221, Yq11.222, Yq11.222-q11.223]","[NLGN4Y, HSFY2, CDY2B, NLGN4Y-AS1, TXLNGY, VCY...",1.679553e-09,[chrY: 13703898.0-19607169.0],SUM159PT
